# Preparation

In [1]:
seed_value= 42

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf
tf.random.set_seed(seed_value)
tf.keras.utils.set_random_seed(seed_value)

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.9.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
num_epochs  = 100
batch_size  = 16
num_classes = 10
shape       = (28, 28, 1)
lr          = 0.0003
opt         = keras.optimizers.Adamax(learning_rate=lr)
los         = keras.losses.CategoricalCrossentropy()
mtr         = ["accuracy"]

# Dataset

In [5]:
def prepare_data(main_path, validation_split=0.1):
    with np.load(main_path) as data:
        x_train, y_train = data['x_train'], data['y_train']
        x_test, y_test = data['x_test'], data['y_test']
        
    # Normalize and reshape the data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    
    x_train = np.reshape(x_train, (-1, 28, 28, 1))
    x_test = np.reshape(x_test, (-1, 28, 28, 1))
    
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    
    # Split the training data into train and validation sets
    val_size = int(len(x_train) * validation_split)
    x_val, y_val = x_train[:val_size], y_train[:val_size]
    x_train, y_train = x_train[val_size:], y_train[val_size:]
    
    # Create TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    
    # Shuffle and batch the datasets
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
    val_dataset = val_dataset.shuffle(buffer_size=1024).batch(batch_size)
    test_dataset = test_dataset.shuffle(buffer_size=1024).batch(batch_size)
    
    # Print the shapes
    print(f"Shape of training images: {x_train.shape}")
    print(f"Shape of validation images: {x_val.shape}")
    print(f"Shape of testing images: {x_test.shape}")
    
    return train_dataset, val_dataset, test_dataset

    
train_dataset, val_dataset, test_dataset = prepare_data(main_path="../Dataset/mnist.npz")    

Shape of training images: (54000, 28, 28, 1)
Shape of validation images: (6000, 28, 28, 1)
Shape of testing images: (10000, 28, 28, 1)


# Model

In [6]:
model = keras.Sequential([keras.layers.InputLayer(shape),
                          layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
                          layers.LeakyReLU(alpha=0.2),
                          layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
                          layers.LeakyReLU(alpha=0.2),
                          layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
                          layers.LeakyReLU(alpha=0.2),
                          layers.GlobalMaxPooling2D(),
                          layers.Dense(64, activation="relu"),
                          layers.Dense(num_classes, activation="softmax")],
                         name="discriminator")

model.compile(optimizer=opt, loss=los, metrics=mtr)
model.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 4, 4, 128)         0         
                                                                 
 global_max_pooling2d (Globa  (None, 128)            

# Train

In [7]:
def callback():
  main_chk  = keras.callbacks.ModelCheckpoint(filepath="Checkpoints/Zero", monitor='val_loss', mode='min', verbose=0, save_best_only=True)
  early_st  = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=30, verbose=0)
  rduce_lr  = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.5, patience=5, verbose=1, min_lr=0.00001)

  return [main_chk, early_st, rduce_lr]

model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs, batch_size=batch_size, callbacks=callback())

Epoch 1/100
3361/3375 [============================>.] - ETA: 0s - loss: 0.5070 - accuracy: 0.8597

INFO:tensorflow:Assets written to: Checkpoints\Zero\assets


INFO:tensorflow:Assets written to: Checkpoints\Zero\assets


3375/3375 [==============================] - 22s 5ms/step - loss: 0.5054 - accuracy: 0.8601 - val_loss: 0.2002 - val_accuracy: 0.9425 - lr: 3.0000e-04
Epoch 2/100
3371/3375 [============================>.] - ETA: 0s - loss: 0.1894 - accuracy: 0.9442

INFO:tensorflow:Assets written to: Checkpoints\Zero\assets


...

INFO:tensorflow:Assets written to: Checkpoints\Zero\assets


3375/3375 [==============================] - 14s 4ms/step - loss: 0.0224 - accuracy: 0.9933 - val_loss: 0.0670 - val_accuracy: 0.9813 - lr: 3.0000e-04
Epoch 25/100
3371/3375 [============================>.] - ETA: 0s - loss: 0.0212 - accuracy: 0.9938

INFO:tensorflow:Assets written to: Checkpoints\Zero\assets


INFO:tensorflow:Assets written to: Checkpoints\Zero\assets


3375/3375 [==============================] - 15s 4ms/step - loss: 0.0212 - accuracy: 0.9938 - val_loss: 0.0644 - val_accuracy: 0.9810 - lr: 3.0000e-04
Epoch 26/100
3375/3375 [==============================] - 13s 4ms/step - loss: 0.0189 - accuracy: 0.9947 - val_loss: 0.0784 - val_accuracy: 0.9775 - lr: 3.0000e-04
Epoch 27/100
3375/3375 [==============================] - 13s 4ms/step - loss: 0.0183 - accuracy: 0.9946 - val_loss: 0.0786 - val_accuracy: 0.9773 - lr: 3.0000e-04
Epoch 28/100
3375/3375 [==============================] - 13s 4ms/step - loss: 0.0167 - accuracy: 0.9951 - val_loss: 0.0851 - val_accuracy: 0.9760 - lr: 3.0000e-04
Epoch 29/100
3375/3375 [==============================] - 12s 4ms/step - loss: 0.0156 - accuracy: 0.9956 - val_loss: 0.0843 - val_accuracy: 0.9768 - lr: 3.0000e-04
Epoch 30/100
3365/3375 [============================>.] - ETA: 0s - loss: 0.0143 - accuracy: 0.9958
Epoch 30: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
3375/3375 [=====

# Evaluation

In [8]:
test_model = tf.keras.models.load_model("Checkpoints/Zero")
test_model.evaluate(test_dataset, verbose=1, batch_size=batch_size)

625/625 [==============================] - 2s 3ms/step - loss: 0.0697 - accuracy: 0.9783


[0.06968142092227936, 0.9782999753952026]